## Trabajo práctico Integrador

**Alumno:** Torresetti Lisandro

**Padrón:** 99846

**Objetivo:**

Realizar un programa que permita determiar la pose de las piezas dispuestas en la zona de trabajo de manera que el robot pueda tomarlas.

In [ ]:
# Imports
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import PIL.ExifTags
import PIL.Image
import pprint
import re
%matplotlib inline

# Paths

BLOCKS_PATH = './imagenes/img_bloques'
BLOCKS_CHALLENGE_PATH = './imagenes/img_bloques_desafio'
CALIBRATION_SET_1 = './imagenes/img_cal_set1'
CALIBRATION_SET_2 = './imagenes/img_cal_set2'

# Constants

EXTRA_IMG = 'imgCalExtr'
IMG_NAME = 'imgBloque'
CALIBRATION_IMG_NAME = 'img_cal1'

# Debug
IMGS_DEBUG = ['imgBloque1', 'imgBloque2', 'imgBloque3', 'imgBloque4', 'imgBloque16']
CHALLENGE_IMGS = IMGS_DEBUG[:3]

## Funciones Auxiliares

In [ ]:
# Auxiliary function to make the plots
def plotter(image, title = '', imgSize = (16,9), grayScale = False):
    plt.figure(figsize=imgSize)
    plt.title(title, fontsize = 16, fontweight = "bold")
    plt.imshow(image) if not grayScale else plt.imshow(image, cmap='gray', vmin=0, vmax=255)
    plt.show()

# Auxiliary function to load all the images from path
def loadImages(path):
    imgNames = glob(path + '/*')
    result = {}
    for imgName in imgNames:
        img = cv.imread(imgName)
        result[re.sub(r'^.*/([^.]*).*$', r'\1', imgName)] = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    return result

# Creates a mask from the image and the specified position. Remember: in OpenCv the coorinates are (Y, X)
def createMask(img, position, lowerMultiplier = 6, upperMultiplier = 6):
    initialPoint, endPoint = position
    colorMean, colorStd = cv.meanStdDev(img[initialPoint[1]:endPoint[1], initialPoint[0]:endPoint[0], :])
    return cv.inRange(img, colorMean - colorStd * lowerMultiplier,  colorMean + colorStd * upperMultiplier)

# Applies a mask to the img. Returns a new image with the mask applied
def applyMask(img, mask):
    return cv.bit

# Plot the histogram for the first 'amountOfBlocks' blocks
def plotHistograms(imgs, bins = 50, amountOfBlocks = 5):
    fig, axs = plt.subplots(amountOfBlocks)
    fig.suptitle('Histograms', fontsize=18, fontweight='bold')
    imgsNames = [IMG_NAME + str(i) for i in range(1, amountOfBlocks + 1)]
    xTicks = np.arange(0, 260, 10)
    yTicks = np.arange(0, 300000, 30000)
    for imgNum, imgName in enumerate(imgsNames):
        axs[imgNum].set_title(imgName, fontsize = 16, fontweight='bold')
        axs[imgNum].grid()
        axs[imgNum].hist(imgs[imgName].ravel(),bins,[0,256], color='orange')
        axs[imgNum].set_xticks(xTicks)
        axs[imgNum].set_yticks(yTicks)
    
    fig.set_size_inches(16, 10)
    fig.tight_layout(pad=5.0)
    
# Performs the Otsu binarization mehtod. Returns a dictionary with the images binarized
def otsuBinarization(imgs, thresh = 100):
    result = {}
    for imgName, img in imgs.items():
        ret, imgBin = cv.threshold(img, thresh, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)
        result[imgName] = imgBin
    return result

# Prints the contour info for an image
def printBlocksInfo(contoursInfo):
    print('RESULTS')
    for blockName, info in contoursInfo.items():
        if isSpecialImage(blockName):
            for block_i_info in info:
                printInfo(blockName, block_i_info)
            continue
            
        printInfo(blockName, info)

# Prints the info of a block with a format
def printInfo(blockName, info):
    print(f'''
                Block Num: {blockName}

                    Area: {info[AREA]}
                    Perimeter: {info[PERIMETER]}
                    Width: {np.round(info[SIDES][0], 4)}
                    Height: {np.round(info[SIDES][1], 4)}
                    Centroid: {info[CENTER]}
                    Aspect Relation: {info[ASPECT_RELATION]}
                    Orientation: {info[ORIENTATION][2]}
                '''
    )

# Returns if the image is the one who contains more than one block
def isSpecialImage(imgName):
    return imgName == IMG_NAME + '16'

# Draws the bounding box and the center of the block. Returns an image with the draws
def drawContours(img, blockInfo):
    outputImg = img.copy()
    contour = blockInfo[CONTOUR]
    cv.circle(outputImg, blockInfo[CENTER], radius=5, color=(0, 0, 0), thickness=-3)
    approx = cv.approxPolyDP(contour, 0.1 * cv.arcLength(contour, True), True)
    cv.drawContours(outputImg, [approx], 0, (255, 0, 0), 2)
    return outputImg

# For debugging
def debug(imgs, grayScale=False, debugImgs=IMGS_DEBUG):
    for imgName in debugImgs:
        plotter(imgs[imgName], imgName, grayScale=grayScale)

## Preprocesamiento

Se cargan las imágenes de los bloques y las de los sets de calibración. Una vez que se carguen todas se procede a preprocesarlas para corregirlas y eliminar el ruido que posean y que pueda interferir en el análisis de las futuras secciones.

In [ ]:
originalBlocks = loadImages(BLOCKS_PATH)
calibrationSet1 = loadImages(CALIBRATION_SET_1)
calibrationSet2 = loadImages(CALIBRATION_SET_2)

# There is an image for the extrinsic calibration, I will store it in some variable and 
# then it will be delete of the set of block images
extrinsicCalibration = originalBlocks[EXTRA_IMG]
del originalBlocks[EXTRA_IMG]

# Just for check if its all right
debug(originalBlocks)

A continuación se crea una máscara con el tablero verde para poder eliminar el ruido de los otros elementos que se encuentran en la imagen. Para crearla, se toma una porción de 'imgBloque1' y otra de 'imgBloque2' para luego hacer un OR entre ambas y obtener una máscara que sólo abarque el tablero verde. De esta forma todos los elementos que no se encuentren en el tablero serán 'eliminados'.

In [ ]:
# Position = ((x1, y1), (x2, y2))
maskPosition1 = ((125, 280), (480, 450))
blocksMask1 = createMask(originalBlocks['imgBloque1'], maskPosition1)

maskPosition2 = ((125, 50), (480, 250))
blocksMask2 = createMask(originalBlocks['imgBloque4'], maskPosition2)
blocksMask = cv.bitwise_or(blocksMask1, blocksMask2)
blocksMask = cv.medianBlur(blocksMask, 5)

blocksWithMask = {}
for imgName, img in originalBlocks.items():
    img = cv.medianBlur(img, 5)
    blocksWithMask[imgName] = cv.bitwise_and(img, img, mask=blocksMask)

# DEBUG
debug(blocksWithMask)

Para obtener una imagen que solo posea la figura del bloque se vuelve a crear una máscara, pero esta vez con porciones de los bloques, para hacer un promedio entre la media y el desvío estandar de cada uno de ellos, con el fin de que sea más general la máscara y no se base en los datos de un solo bloque. Se utilizarán los bloques de las imágenes 1, 2 y 3 para obtener la máscara deseada.

In [ ]:
# Blocks Positions
block1 = ((325, 110), (400, 125))
block2 = ((285, 150), (325, 195))
block3 = ((400, 150), (450, 200))
blocks = [block1, block2, block3]

# Draw a red line to verify the positions
for blockNum, block in enumerate(blocks):
    testImg = blocksWithMask[IMG_NAME + str(blockNum + 1)].copy()
    cv.line(testImg, block[0], block[1], (255,0,0), 5)
    plotter(testImg, IMG_NAME + str(blockNum + 1))

# Get statistics about each block to compute the mask
totalMeanBlocks = 0
totalStdBlocks = 0
for blockNum, blockPosition in enumerate(blocks):
    initialPoint, endPoint = blockPosition
    imgName = IMG_NAME + str(blockNum + 1)
    blockImg = blocksWithMask[imgName][initialPoint[1]:endPoint[1], initialPoint[0]:endPoint[0], :]
    meanBlock, stdBlock = cv.meanStdDev(blockImg)
    totalMeanBlocks += meanBlock
    totalStdBlocks += stdBlock
    plotter(blockImg, imgName, imgSize=(8,5))

meanBlocks = totalMeanBlocks / len(blocks)
stdBlocks = totalStdBlocks / len(blocks)

In [ ]:
print(f"Mean: {meanBlocks} \nStd: {stdBlocks}")

A coninuación se aplican los datos obtenidos a todas las imágenes para obtnener sólo la figura de los bloques en cada una de ellas. Los resultados serán guardados en **blocksWithMask** en escala de grises, 'pisando' los valores que se obtuvieron anteriormente.

In [ ]:
for imgName, img in blocksWithMask.items():
    mask_i = cv.inRange(img, meanBlocks - stdBlocks * 6,  meanBlocks + stdBlocks * 6)
    blocksWithMask[imgName] = cv.cvtColor(cv.bitwise_and(img, img, mask=mask_i), cv.COLOR_RGB2GRAY)

debug(blocksWithMask, grayScale=True)

Dado que las imágenes poseen ciertas imperfecciones, como por ejemplo puntos negros dentro de los bloques o problemas en los bordes de los mismos. Se procede a binarizarlas para poder aplicarles operaciónes morfológicas y así corregir estas imperfecciones. Antes de realizar esto, se grafican los histogramas de dos imágenes para ver la distribución de los valores y así poder elegir un valor de _threshold_ adecuado.

In [ ]:
plotHistograms(blocksWithMask, amountOfBlocks=2)

El resultado de los histogramas es el esperado, ya que el porcentaje de color del bloque es pequeño en comparación con toda la imagen que es casi en su totalidad negra. Se utilizará un _threshold_ de 100 para realizar la binarización de las imágenes por el método de **Otsu** que se describe a continuación.

**// ToDo: poner descripcion de otsu**

In [ ]:
binarizedBlocks = otsuBinarization(blocksWithMask)
debug(binarizedBlocks, grayScale=True)

Para solucionar los problemas mencionados y que se encuentran a simple vista, se aplicará la operación morfológica de dilatación con un _Structural element_ de 4x1. Al resultado de esta operación se le aplicará un filtro de mediana con un kernel de 5x5. El resultado de estas operaciones se almacenará en la variable _final blocks_.

In [ ]:
# This dictionary will contain the final result of all the previous operation 
# plus the next one for each block image
kernelDilate =np.ones((4,1), np.uint8)
finalBlocks = {}
for blockName, blockImg in binarizedBlocks.items():
    finalBlocks[blockName] = cv.medianBlur(cv.dilate(blockImg, kernelDilate), 5)

debug(finalBlocks, grayScale=True)

Con esto se da por finalizado el preprocesamiento de las imágenes. Los resultados obtenidos se utilizarán en las siguientes secciones.

## 1 - Calibración Intrínseca

La matriz de parámetros intrínsecos describe todos los parámetros internos de la cámara. La matriz contiene las distancias focales ($f_x$ y $f_y$) y los centros ópticos ($c_x$ y $c_y$) expresados en coordenadas de píxeles. En un modelo ideal $f_x = f_y = f$, sin embargo esto en la realidad estos valores pueden diferir debido a fallas en el sensor de la cámara digital. La matriz de parámetros intrínsecos es la siguiente:
$$\begin{bmatrix} fx & s & cx \\ 0 & fy & cy \\ 0 & 0 & 1 \end{bmatrix}$$

Una cámara puede estar sujeta a distorsiones radiales o tangenciales, llevando a un _fish-eye effect_. Estas distorsiones pueden ser descritas a traves de una lista de _coeficientes de distorción_

Para estimar los parámetros intrínsecos de la cámara se utilizarán los dos sets de calibración (en ambos sets el patrón es de 8x6). Una vez que se obtengan los resultados se los analizará para determinar cuál es el más indicado para el problema.
Se utilizará el método _calibrateCamera_ que provee _openCV_ para hacer las estimaciones correspondientes.

In [ ]:
chessBoardSize  = (8, 6)
objp = np.zeros((np.prod(chessBoardSize), 3),  dtype=np.float32)
objp[:, :2] = np.mgrid[0:chessBoardSize[0], 0:chessBoardSize[1]].T.reshape(-1, 2)

# Finds the corners of the images
def findCorners(imgs, plot=True, maxCount = 25, epsilon = 0.001, flag=cv.CALIB_CB_ADAPTIVE_THRESH):
    imgPoints = []
    objPoints = []
    criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_MAX_ITER, maxCount, epsilon)
    cb_flags = flag 
    for imgName, img in imgs.items():
        img = img.copy()
        imgGray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
        ret, corners = cv.findChessboardCorners(imgGray, chessBoardSize, flags=cb_flags)
        if ret:
            objPoints.append(objp)
            corners_subp = cv.cornerSubPix(imgGray, corners, (5, 5), (-1, -1), criteria)
            imgPoints.append(corners_subp)
            cv.drawChessboardCorners(img, chessBoardSize, corners_subp, ret)
            if plot:
                plotter(img, imgName)
    return imgPoints, objPoints

# Calibrates the camera based on the parameters
def calibrateCamera(objPoints, imgPoints, width, height, returnMatrix=False):
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objPoints, imgPoints, (width, height), None, None)
    print('Camera Matrix: \n{}'.format(mtx))
    print('\nDistortion Coefficients: \n{}\n'.format(dist))
    if returnMatrix:
        return mtx

Con las funciones creadas se procede a calibrar la cámara para los distintos sets

### Set 1

In [ ]:
imgPointsSet1, objPointsSet1 = findCorners(calibrationSet1)

In [ ]:
heightSet1, widthSet1, _ = calibrationSet1[CALIBRATION_IMG_NAME].shape
cameraMatrixSet1 = calibrateCamera(objPointsSet1, imgPointsSet1, widthSet1, heightSet1, returnMatrix=True)

### Set 2

In [ ]:
imgPointsSet2, objPointsSet2 = findCorners(calibrationSet2)

In [ ]:
heightSet2, widthSet2, _ = calibrationSet1[CALIBRATION_IMG_NAME].shape
cameraMatrixSet2 = calibrateCamera(objPointsSet2, imgPointsSet2, widthSet2, heightSet2, returnMatrix=True)

## 2 - Calibración Extrínseca

## 3 - Búsqueda de bloques

Dado que las imagenes de los bloques se encuentran binarizadas y corregidas por el preprocesamiento que se realizo anteriormente, se puede utilizar la función _findContours_ que provee openCV para obtener los contornos de los bloques. Esta función trabaja sobre imágenes binarias y devuelve un conjunto de puntos que se cree que son parte del contorno.

In [ ]:
blockContours = {}

for blockName, blockImg in finalBlocks.items():
    contours, hier = cv.findContours(blockImg, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    blockContours[blockName] = contours
    output = originalBlocks[blockName].copy()
    cv.drawContours(output, contours, -1, (255,0,0),2)
    plotter(output, blockName)

Se puede apreciar que los bordes de los bloques son captados de forma correcta, algunos con ciertas irregularidades pero en la mayoría de los casos se los capta bien. A continuación se escriben funciones auxiliares para obtener los datos de los contornos de los bloques (área, lados, centroides, etc).

In [ ]:
def getAspectRatio(contour):
    x,y,width,height = cv.boundingRect(contour)
    return np.round(float(width) / height, 2)

# Returns the centroids (cx, cy)
def getCentroid(moment, area):
    return (int(moment['m10'] / area), int(moment['m01'] / area))

# Returns the angle in degrees
def getOrientation(contour):
    (x, y), (MA, mA), angle = cv.fitEllipse(contour)
    return (int(x), int(y)), (int(MA), int(mA)), np.round(angle)

# Returnrs (width, height) of the rectangle
def getSides(contour):
    return cv.minAreaRect(contour)[1]

In [ ]:
# More info here: https://en.wikipedia.org/wiki/Image_moment
# Constants to get the info easily
AREA = 'area'
PERIMETER = 'perimeter'
CENTER = 'center'
SIDES = 'sides'
ASPECT_RELATION = 'aspectRelation'
ORIENTATION = 'orientation'
CONTOUR = 'contour'
MIN_AREA = 8000 # According to the TP, the area is 8450

def getContoursInfo(blockContours):
    contoursInfo = {}
    
    for blockName, contours in blockContours.items():
        for ctr in contours:
            information = {}
            moment = cv.moments(ctr)
            area = int(moment['m00'])
            if area <= MIN_AREA: # Is not a contour of a block
                continue
            information[AREA] = area
            information[PERIMETER] = int(cv.arcLength(ctr,True))
            information[CENTER] = getCentroid(moment, area)
            information[SIDES] = getSides(ctr)
            information[ASPECT_RELATION] = getAspectRatio(ctr)
            information[ORIENTATION] = getOrientation(ctr)
            information[CONTOUR] = ctr
            if isSpecialImage(blockName):
                # This image is special because there are two blocks
                contoursInfo[blockName] = contoursInfo.get(blockName, []) + [information]
                continue
            contoursInfo[blockName] = information
            
    return contoursInfo

contoursInfo = getContoursInfo(blockContours) 

A continuación para cada una de las imágenes analizadas se marcará en rojo los bordes del bloque y con un punto negro dónde se detectó el centro del mismo.

In [ ]:
for blockName, info in contoursInfo.items():
    if isSpecialImage(blockName):
            finalImg = originalBlocks[blockName].copy()
            for block_i_info in info:
                finalImg = drawContours(finalImg, block_i_info)
            plotter(finalImg, blockName)
            continue
    plotter(drawContours(originalBlocks[blockName], info), blockName)

## 4 - Validación del algoritmo

## 5 - Medición de bloques

## 6 - Challenge

Se realizará lo mismo que en las secciones anteriores, solo que esta vez aplicando todo a los bloques del desafío. Primero se cargan las imágenes en una nueva variable y luego se realiza un preprocesamiento para eliminar todo el ruído que puede afectar al análisis de los bloques.

In [ ]:
challengeBlocks = loadImages(BLOCKS_CHALLENGE_PATH)
extrinsicCalibrationChallenge = challengeBlocks[EXTRA_IMG]
del challengeBlocks[EXTRA_IMG]
debug(challengeBlocks, debugImgs=CHALLENGE_IMGS)

Como se puede apreciar, estas imágenes poseen mucho más ruido que las anteriores, por ejemplo se puede ver un cable azul que atraviesa la escena, o una botella de gaseosa y partes de otros bloques que se encuentran afuera del tablero verde. A continuación se utiliza las máscaras creadas anteriormente para ver si con esto se solucionan estos problemas.

In [ ]:
# Apply board mask
challengeBlocksWithMask = {}
for imgName, img in challengeBlocks.items():
    img = cv.medianBlur(img - 50, 5) # In order to reduce the bright of each image
    challengeBlocksWithMask[imgName] = cv.bitwise_and(img, img, mask=blocksMask)

# DEBUG
debug(challengeBlocksWithMask, debugImgs=CHALLENGE_IMGS)

In [ ]:
# Apply block mask and change imgs to gray scale
for imgName, img in challengeBlocksWithMask.items():
    mask_i = cv.inRange(img, meanBlocks - stdBlocks * 17,  meanBlocks + stdBlocks * 6)
    challengeBlocksWithMask[imgName] = cv.cvtColor(cv.bitwise_and(img, img, mask=mask_i), cv.COLOR_RGB2GRAY)

debug(challengeBlocksWithMask, grayScale=True, debugImgs=CHALLENGE_IMGS)

Como se puede apreciar, bajando el brillo de las imágenes y aplicando las mismas máscaras que antes se pueden obtener bien las formas de los bloques. Dado que sigue habiendo ruido en las imágenes se las binarizá para luego aplicarles operaciones morfológicas para eliminar la mayor cantidad de ruido que sea posible. Al igual que en el preprocesamiento anterior, se graficarán los histogramas de las 3 imágenes para poder establecer el _threshold_ a utilizar en el método de binarización de Otsu.

In [ ]:
plotHistograms(challengeBlocksWithMask, amountOfBlocks=3)

Nuevamente este resultado es el esperado, dado que el porcentaje de negro en cada una de las imágenes es mucho mayor que el porcentaje gris que representa a los bloques. Se utiliza el mismo threshold que en el caso anterior para binarizar a las imágenes.

In [ ]:
binarizedChallengeBlocks = otsuBinarization(challengeBlocksWithMask)
debug(binarizedChallengeBlocks, grayScale=True, debugImgs=CHALLENGE_IMGS)

Se aplicará la operación morfológica de erosión con un _Structural Element_ de 3x3 un total de 3 veces y luego la operación de dilatación con un structural element del mismo tamaño, pero un total de 4 veces. Por último, al resultado de esas operaciones se les aplica un filtro de mediana con un kernel de 5x5. El resultado de todas estas operaciones se guarda en la variable **finalChallengeBlocks** que se utilizará más adelante para buscar los contornos y propiedades de los bloques.

In [ ]:
kernelErotion = np.ones((3,3), np.uint8)
kernelDilation = np.ones((3,3), np.uint8)
finalChallengeBlocks = {}
for imgName, binarizedBlock in binarizedChallengeBlocks.items():
    img = binarizedBlock.copy()
    img = cv.erode(img, kernelErotion, iterations=3)
    img = cv.dilate(img, kernelDilation, iterations=4)
    img = cv.medianBlur(img, 5)
    finalChallengeBlocks[imgName] = img
    plotter(img, imgName, grayScale=True)


In [ ]:
challengeBlockContours = {}

for blockName, blockImg in finalChallengeBlocks.items():
    contours, hier = cv.findContours(blockImg, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    challengeBlockContours[blockName] = contours
    output = challengeBlocks[blockName].copy()
    cv.drawContours(output, contours, -1, (255,0,0),2)
    plotter(output, blockName)

In [ ]:
contoursInfo = getContoursInfo(challengeBlockContours)

In [ ]:
for blockName, info in contoursInfo.items():
    if isSpecialImage(blockName):
            finalImg = challengeBlocks[blockName].copy()
            for block_i_info in info:
                finalImg = drawContours(finalImg, block_i_info)
            plotter(finalImg, blockName)
            continue
    plotter(drawContours(challengeBlocks[blockName], info), blockName)

In [ ]:
printBlocksInfo(contoursInfo)

Se puede apreciar que los contornos fueron hallados de forma exitosa así como las distintas propiedades de los bloques.